In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

from NegativeClassOptimization import utils, preprocessing, ml, config, datasets, visualisations

/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# ID

In [2]:
def get_id_results(dir_path, fname):
    base_p = Path(config.DATA_BASE_PATH) / dir_path
    df = pd.read_csv(base_p / fname, sep='\t')

    records = []
    for i, row in df.iterrows():
        task_str = row["task"]
        task = datasets.ClassificationTask.init_from_str(task_str)
        records.append({
            "task": task_str,
            "ag_pos": task.ag_pos,
            "ag_neg": task.ag_neg,
            "seed_id": task.seed_id,
            "split_id": task.split_id,
            "task_type": task.task_type.to_str(),
        })

    df = pd.merge(df, pd.DataFrame(records), on="task")

    map_task_type_to_clean = visualisations.PlotParams.map_task_type_to_clean.copy()
    df["task_type_clean"] = df["task_type"].map(map_task_type_to_clean)
    df["acc_closed"] = df["acc"].astype(float)
    return df

In [3]:
df_1 = get_id_results("Frozen_MiniAbsolut_ML", "closed_performance.tsv")
df_1["setup"] = "SN10"

df_2 = get_id_results("Frozen_MiniAbsolut_Linear_ML", "closed_performance_logistic.tsv")
df_2["setup"] = "Logistic"

df_3 = get_id_results("Frozen_MiniAbsolut_ML_Transformer_Parameters_Absolut/transformer_parameterset_a764f9c7", "closed_performance.tsv")
df_3["setup"] = "Transformer - Absolut"

df_3b = get_id_results("Frozen_MiniAbsolut_ML_Transformer_Parameters_Experimental/transformer_parameterset_085feb71", "closed_performance.tsv")
df_3b["setup"] = "Transformer - Experimental"

df_4 = get_id_results("Frozen_MiniAbsolut_ML_x10under", "closed_performance.tsv")
df_4["setup"] = "SN10 x10_undersampling"

df_5 = get_id_results("Frozen_MiniAbsolut_ML_x50under", "closed_performance.tsv")
df_5["setup"] = "SN10 x50_undersampling"

df_6 = get_id_results("Frozen_MiniAbsolut_ML_antiberta2", "closed_performance.tsv")
df_6["setup"] = "PLM Antiberta2"

df_6b = get_id_results("Frozen_MiniAbsolut_ML_antiberta2_experimental", "closed_performance.tsv")
df_6b["setup"] = "PLM Antiberta2 - Experimental"

df_7 = get_id_results("Frozen_MiniAbsolut_ML_esm2b", "closed_performance.tsv")
df_7["setup"] = "PLM ESM2b"

df_7b = get_id_results("Frozen_MiniAbsolut_ML_esm2b_experimental", "closed_performance.tsv")
df_7b["setup"] = "PLM ESM2b - Experimental"

df = pd.concat([df_1, df_2, df_3, df_3b, df_4, df_5, df_6, df_6b, df_7, df_7b], axis=0)
df = df.loc[df["task_type_clean"] != "vs 1"].copy()
df.head()

,task,acc,recall,precision,f1,ag_pos,ag_neg,seed_id,split_id,task_type,task_type_clean,acc_closed,setup,N_pos,N_neg
0,HIGH_VS_95LOW__3VRL__auto__0__42,0.996600,0.9980,0.995213,0.996605,3VRL,auto,0,42,high_vs_95low,vs Non-binder,0.996600,SN10,NaN,NaN
1,HIGH_VS_LOOSER__3VRL__auto__0__42,0.953400,0.9604,0.947140,0.953724,3VRL,auto,0,42,high_vs_looser,vs Weak,0.953400,SN10,NaN,NaN
2,ONE_VS_NINE__3VRL__auto__0__42,0.972986,0.9750,0.971116,0.973054,3VRL,auto,0,42,1v9,vs 9,0.972986,SN10,NaN,NaN
3,HIGH_VS_95LOW__3VRL__auto__1__42,0.996200,0.9986,0.993830,0.996209,3VRL,auto,1,42,high_vs_95low,vs Non-binder,0.996200,SN10,NaN,NaN
4,HIGH_VS_LOOSER__3VRL__auto__1__42,0.952600,0.9576,0.948119,0.952836,3VRL,auto,1,42,high_vs_looser,vs Weak,0.952600,SN10,NaN,NaN


In [4]:
df.groupby(["setup", "task_type_clean"])["acc_closed"].median().reset_index().pivot(columns=["setup"], values=["acc_closed"], index=["task_type_clean"])

acc_closed                                               \
setup             Logistic PLM Antiberta2 PLM Antiberta2 - Experimental   
task_type_clean                                                           
vs 9              0.864682        0.82963                           NaN   
vs Non-binder     0.971650        0.95430                         0.883   
vs Weak           0.791550        0.76975                         0.750   

                                                              \
setup           PLM ESM2b PLM ESM2b - Experimental      SN10   
task_type_clean                                                
vs 9             0.874875                      NaN  0.944422   
vs Non-binder    0.974900                    0.901  0.988000   
vs Weak          0.811100                    0.742  0.917350   

                                                               \
setup           SN10 x10_undersampling SN10 x50_undersampling   
task_type_clean                                                 
vs 9                          0.890891                  0.815   
vs Non-binder                 0.963000                  0.940   
vs Weak                       0.823500                  0.675   

                                                                  
setup           Transformer - Absolut Transformer - Experimental  
task_type_clean                                                   
vs 9                          0.86190                        NaN  
vs Non-binder                 0.95635                      0.962  
vs Weak                       0.65585                      0.827

In [5]:
df.groupby(["setup", "task_type_clean"])["acc_closed"].apply(lambda x: f"{x.median():.2f} ({x.quantile(0.25):.2f} - {x.quantile(0.75):.2f})").reset_index().pivot(columns=["setup"], values=["acc_closed"], index=["task_type_clean"]).transpose()

task_type_clean                                         vs 9  \
           setup                                               
acc_closed Logistic                       0.86 (0.85 - 0.91)   
           PLM Antiberta2                 0.83 (0.81 - 0.86)   
           PLM Antiberta2 - Experimental                 NaN   
           PLM ESM2b                      0.87 (0.86 - 0.90)   
           PLM ESM2b - Experimental                      NaN   
           SN10                           0.94 (0.94 - 0.96)   
           SN10 x10_undersampling         0.89 (0.87 - 0.92)   
           SN10 x50_undersampling         0.81 (0.78 - 0.85)   
           Transformer - Absolut          0.86 (0.85 - 0.87)   
           Transformer - Experimental                    NaN   

task_type_clean                                vs Non-binder  \
           setup                                               
acc_closed Logistic                       0.97 (0.96 - 0.99)   
           PLM Antiberta2                 0.95 (0.94 - 0.97)   
           PLM Antiberta2 - Experimental  0.88 (0.88 - 0.88)   
           PLM ESM2b                      0.97 (0.97 - 0.99)   
           PLM ESM2b - Experimental       0.90 (0.90 - 0.90)   
           SN10                           0.99 (0.98 - 1.00)   
           SN10 x10_undersampling         0.96 (0.95 - 0.98)   
           SN10 x50_undersampling         0.94 (0.92 - 0.95)   
           Transformer - Absolut          0.96 (0.94 - 0.98)   
           Transformer - Experimental     0.96 (0.95 - 0.96)   

task_type_clean                                      vs Weak  
           setup                                              
acc_closed Logistic                       0.79 (0.77 - 0.86)  
           PLM Antiberta2                 0.77 (0.74 - 0.78)  
           PLM Antiberta2 - Experimental  0.75 (0.75 - 0.75)  
           PLM ESM2b                      0.81 (0.79 - 0.85)  
           PLM ESM2b - Experimental       0.74 (0.74 - 0.74)  
           SN10                           0.92 (0.89 - 0.94)  
           SN10 x10_undersampling         0.82 (0.80 - 0.87)  
           SN10 x50_undersampling         0.68 (0.65 - 0.70)  
           Transformer - Absolut          0.66 (0.63 - 0.70)  
           Transformer - Experimental     0.83 (0.80 - 0.83)

# OOD

In [6]:
def get_ood_results(dir_path, fname):
    base_p = Path(config.DATA_BASE_PATH) / dir_path
    df_o = pd.read_csv(base_p / fname, sep='\t')

    df_o["ag"] = df_o["task_1"].apply(lambda x: datasets.ClassificationTask.init_from_str(x).ag_pos)
    df_o["task_1_type"] = df_o["task_1"].apply(lambda x: datasets.ClassificationTask.init_from_str(x).task_type.to_str())
    df_o["task_2_type"] = df_o["task_2"].apply(lambda x: datasets.ClassificationTask.init_from_str(x).task_type.to_str())
    def get_replicate_id(task_str):
        task = datasets.ClassificationTask.init_from_str(task_str)
        return str(task.seed_id) + "_" + str(task.split_id)
    df_o["replicate_id"] = df_o["task_1"].apply(get_replicate_id)

    df_o = df_o.query("task_1_type != '1v1' and task_2_type != '1v1'").copy()

    df_o["model_task"] = df_o["task_1_type"].apply(lambda x: visualisations.PlotParams.map_task_type_to_clean[x])
    df_o["test_task"] = df_o["task_2_type"].apply(lambda x: visualisations.PlotParams.map_task_type_to_clean[x])

    return df_o

In [7]:
df_1 = get_ood_results("Frozen_MiniAbsolut_ML", "openset_performance.tsv")
df_1["setup"] = "SN10"

df_2 = get_ood_results("Frozen_MiniAbsolut_Linear_ML", "openset_performance_logistic.tsv")
df_2["setup"] = "Logistic"

df_3 = get_ood_results("Frozen_MiniAbsolut_ML_Transformer_Parameters_Absolut/transformer_parameterset_a764f9c7", "openset_performance.tsv")
df_3["setup"] = "Transformer - Absolut"

df_3b = get_ood_results("Frozen_MiniAbsolut_ML_Transformer_Parameters_Experimental/transformer_parameterset_085feb71", "openset_performance.tsv")
df_3b["setup"] = "Transformer - Experimental"

df_4 = get_ood_results("Frozen_MiniAbsolut_ML_x10under", "openset_performance.tsv")
df_4["setup"] = "SN10 x10_undersampling"

df_5 = get_ood_results("Frozen_MiniAbsolut_ML_x50under", "openset_performance.tsv")
df_5["setup"] = "SN10 x50_undersampling"

df_6 = get_ood_results("Frozen_MiniAbsolut_ML_antiberta2", "openset_performance.tsv")
df_6["setup"] = "PLM Antiberta2"

df_6b = get_ood_results("Frozen_MiniAbsolut_ML_antiberta2_experimental", "openset_performance.tsv")
df_6b["setup"] = "PLM Antiberta2 - Experimental"

df_7 = get_ood_results("Frozen_MiniAbsolut_ML_esm2b", "openset_performance.tsv")
df_7["setup"] = "PLM ESM2b"

df_7b = get_ood_results("Frozen_MiniAbsolut_ML_esm2b_experimental", "openset_performance.tsv")
df_7b["setup"] = "PLM ESM2b - Experimental"

df = pd.concat([df_1, df_2, df_3, df_3b, df_4, df_5, df_6, df_6b, df_7, df_7b], axis=0)
# df = df.loc[df["task_type_clean"] != "vs 1"].copy()
df.head()

,task_1,task_2,acc,recall,precision,f1,ag,task_1_type,task_2_type,replicate_id,model_task,test_task,setup
0,HIGH_VS_95LOW__3VRL__auto__0__42,HIGH_VS_95LOW__3VRL__auto__0__42,0.996600,0.998,0.995213,0.996605,3VRL,high_vs_95low,high_vs_95low,0_42,vs Non-binder,vs Non-binder,SN10
1,HIGH_VS_95LOW__3VRL__auto__0__42,HIGH_VS_LOOSER__3VRL__auto__0__42,0.758500,0.998,0.674780,0.805163,3VRL,high_vs_95low,high_vs_looser,0_42,vs Non-binder,vs Weak,SN10
2,HIGH_VS_95LOW__3VRL__auto__0__42,ONE_VS_NINE__3VRL__auto__0__42,0.912356,0.998,0.852117,0.919307,3VRL,high_vs_95low,1v9,0_42,vs Non-binder,vs 9,SN10
3,HIGH_VS_95LOW__3VRL__auto__0__42,HIGH_VS_95LOW__3VRL__auto__1__42,0.996600,0.998,0.995213,0.996605,3VRL,high_vs_95low,high_vs_95low,0_42,vs Non-binder,vs Non-binder,SN10
4,HIGH_VS_95LOW__3VRL__auto__0__42,HIGH_VS_LOOSER__3VRL__auto__1__42,0.758500,0.998,0.674780,0.805163,3VRL,high_vs_95low,high_vs_looser,0_42,vs Non-binder,vs Weak,SN10


In [8]:
df.groupby(["setup", "model_task", "test_task"])["acc"].median().reset_index().pivot(columns=["setup"], values=["acc"], index=["model_task", "test_task"])

acc                 \
setup                        Logistic PLM Antiberta2   
model_task    test_task                                
vs 9          vs 9           0.865583       0.829630   
              vs Non-binder  0.790550       0.844800   
              vs Weak        0.636600       0.663200   
vs Non-binder vs 9           0.705653       0.638939   
              vs Non-binder  0.972700       0.954300   
              vs Weak        0.659800       0.626800   
vs Weak       vs 9           0.779390       0.723574   
              vs Non-binder  0.896700       0.887200   
              vs Weak        0.793200       0.769750   

                                                                     \
setup                       PLM Antiberta2 - Experimental PLM ESM2b   
model_task    test_task                                               
vs 9          vs 9                                    NaN  0.879680   
              vs Non-binder                           NaN  0.847500   
              vs Weak                                 NaN  0.667300   
vs Non-binder vs 9                                    NaN  0.663764   
              vs Non-binder                         0.883  0.973500   
              vs Weak                               0.654  0.680200   
vs Weak       vs 9                                    NaN  0.752953   
              vs Non-binder                         0.795  0.886200   
              vs Weak                               0.750  0.815900   

                                                                \
setup                       PLM ESM2b - Experimental      SN10   
model_task    test_task                                          
vs 9          vs 9                               NaN  0.949675   
              vs Non-binder                      NaN  0.917400   
              vs Weak                            NaN  0.714250   
vs Non-binder vs 9                               NaN  0.812206   
              vs Non-binder                    0.901  0.989100   
              vs Weak                          0.660  0.723650   
vs Weak       vs 9                               NaN  0.904352   
              vs Non-binder                    0.801  0.960500   
              vs Weak                          0.742  0.921150   

                                                                           \
setup                       SN10 x10_undersampling SN10 x50_undersampling   
model_task    test_task                                                     
vs 9          vs 9                        0.891892                 0.8000   
              vs Non-binder               0.826000                 0.6675   
              vs Weak                     0.659000                 0.5925   
vs Non-binder vs 9                        0.719720                 0.6650   
              vs Non-binder               0.964000                 0.9125   
              vs Weak                     0.663000                 0.5950   
vs Weak       vs 9                        0.818819                 0.7225   
              vs Non-binder               0.902000                 0.8075   
              vs Weak                     0.824000                 0.6575   

                                                                              
setup                       Transformer - Absolut Transformer - Experimental  
model_task    test_task                                                       
vs 9          vs 9                        0.86070                        NaN  
              vs Non-binder               0.62950                        NaN  
              vs Weak                     0.54530                        NaN  
vs Non-binder vs 9                        0.67620                        NaN  
              vs Non-binder               0.95795                      0.960  
              vs Weak                     0.63925                      0.667  
vs Weak       vs 9                        0.65075                        NaN  


In [9]:
df.groupby(["setup", "model_task", "test_task"])["acc"].apply(lambda x: f"{x.median():.2f} ({x.quantile(0.25):.2f} - {x.quantile(0.75):.2f})").reset_index().pivot(columns=["setup"], values=["acc"], index=["model_task", "test_task"]).transpose()

model_task                                       vs 9                      \
test_task                                        vs 9       vs Non-binder   
    setup                                                                   
acc Logistic                       0.87 (0.85 - 0.91)  0.79 (0.66 - 0.90)   
    PLM Antiberta2                 0.83 (0.81 - 0.86)  0.84 (0.77 - 0.91)   
    PLM Antiberta2 - Experimental                 NaN                 NaN   
    PLM ESM2b                      0.88 (0.86 - 0.90)  0.85 (0.66 - 0.90)   
    PLM ESM2b - Experimental                      NaN                 NaN   
    SN10                           0.95 (0.94 - 0.96)  0.92 (0.81 - 0.98)   
    SN10 x10_undersampling         0.89 (0.87 - 0.92)  0.83 (0.73 - 0.90)   
    SN10 x50_undersampling         0.80 (0.79 - 0.82)  0.67 (0.65 - 0.68)   
    Transformer - Absolut          0.86 (0.84 - 0.88)  0.63 (0.51 - 0.77)   
    Transformer - Experimental                    NaN                 NaN   

model_task                                                  vs Non-binder  \
test_task                                     vs Weak                vs 9   
    setup                                                                   
acc Logistic                       0.64 (0.62 - 0.72)  0.71 (0.69 - 0.77)   
    PLM Antiberta2                 0.66 (0.63 - 0.69)  0.64 (0.61 - 0.67)   
    PLM Antiberta2 - Experimental                 NaN                 NaN   
    PLM ESM2b                      0.67 (0.66 - 0.69)  0.66 (0.64 - 0.74)   
    PLM ESM2b - Experimental                      NaN                 NaN   
    SN10                           0.71 (0.68 - 0.80)  0.81 (0.80 - 0.83)   
    SN10 x10_undersampling         0.66 (0.63 - 0.71)  0.72 (0.70 - 0.75)   
    SN10 x50_undersampling         0.59 (0.55 - 0.63)  0.67 (0.64 - 0.67)   
    Transformer - Absolut          0.55 (0.54 - 0.59)  0.68 (0.54 - 0.82)   
    Transformer - Experimental                    NaN                 NaN   

model_task                                                                 \
test_task                               vs Non-binder             vs Weak   
    setup                                                                   
acc Logistic                       0.97 (0.96 - 0.99)  0.66 (0.65 - 0.68)   
    PLM Antiberta2                 0.95 (0.94 - 0.97)  0.63 (0.62 - 0.64)   
    PLM Antiberta2 - Experimental  0.88 (0.88 - 0.88)  0.65 (0.65 - 0.65)   
    PLM ESM2b                      0.97 (0.96 - 0.98)  0.68 (0.67 - 0.70)   
    PLM ESM2b - Experimental       0.90 (0.90 - 0.90)  0.66 (0.66 - 0.66)   
    SN10                           0.99 (0.98 - 1.00)  0.72 (0.70 - 0.74)   
    SN10 x10_undersampling         0.96 (0.95 - 0.98)  0.66 (0.65 - 0.68)   
    SN10 x50_undersampling         0.91 (0.90 - 0.92)  0.59 (0.58 - 0.61)   
    Transformer - Absolut          0.96 (0.94 - 0.98)  0.64 (0.61 - 0.66)   
    Transformer - Experimental     0.96 (0.93 - 0.96)  0.67 (0.66 - 0.68)   

model_task                                    vs Weak                      \
test_task                                        vs 9       vs Non-binder   
    setup                                                                   
acc Logistic                       0.78 (0.75 - 0.88)  0.90 (0.88 - 0.93)   
    PLM Antiberta2                 0.72 (0.68 - 0.74)  0.89 (0.87 - 0.90)   
    PLM Antiberta2 - Experimental                 NaN  0.80 (0.80 - 0.80)   
    PLM ESM2b                      0.75 (0.74 - 0.78)  0.89 (0.88 - 0.91)   
    PLM ESM2b - Experimental                      NaN  0.80 (0.80 - 0.80)   
    SN10                           0.90 (0.86 - 0.94)  0.96 (0.95 - 0.97)   
    SN10 x10_undersampling         0.82 (0.78 - 0.90)  0.90 (0.88 - 0.93)   
    SN10 x50_undersampling         0.72 (0.69 - 0.74)  0.81 (0.80 - 0.83)   
    Transformer - Absolut          0.65 (0.58 - 0.75)  0.88 (0.85 - 0.92)   
    Transformer - Experimental                    NaN  0.60 (0.52 - 0.